In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, sum
import pandas as pd
import numpy as np

In [67]:
spark = SparkSession.builder.appName("sparkWindowFunctionsExample").getOrCreate()
data = [(1,100,1,2022),(1,700,2,2022),(1,600,2,2022),(2,1000,3,2022),(2,1100,1,2022),(3,400,1,2021),(3,500,1,2021)]
df = spark.createDataFrame(data,schema=["id","salary","month","year"])
df.createOrReplaceTempView("df")
window_1=Window.partitionBy(["year","month"]).orderBy("salary")
df=df.withColumn("ytd_sum",sum("salary").over(window_1)).drop("id")
df.show()

+------+-----+----+-------+
|salary|month|year|ytd_sum|
+------+-----+----+-------+
|   400|    1|2021|    400|
|   500|    1|2021|    900|
|   100|    1|2022|    100|
|  1100|    1|2022|   1200|
|   600|    2|2022|    600|
|   700|    2|2022|   1300|
|  1000|    3|2022|   1000|
+------+-----+----+-------+



In [68]:
df2=spark.sql("""SELECT
    salary,
    month,
    year,
    SUM(salary) OVER (PARTITION BY year,month ORDER BY salary) AS ytd_sum
FROM df
ORDER BY year, month;""")
df2.show()

+------+-----+----+-------+
|salary|month|year|ytd_sum|
+------+-----+----+-------+
|   400|    1|2021|    400|
|   500|    1|2021|    900|
|   100|    1|2022|    100|
|  1100|    1|2022|   1200|
|   600|    2|2022|    600|
|   700|    2|2022|   1300|
|  1000|    3|2022|   1000|
+------+-----+----+-------+



In [71]:
df_dict={"id":[1,1,1,2,2,3,3],"salary":[100,700,600,1000,1100,400,500],"month":["1","2","2","3","1","1","1"],
    "year":[2022,2022,2022,2022,2022,2021,2021]}
df=pd.DataFrame(df_dict)
display(df)
df["YTD Sum"]=df[["year","month","salary"]].groupby(["year","month"])["salary"].cumsum()
df.sort_values(["year","month"]).drop("id",axis=1)

,id,salary,month,year
0,1,100,1,2022
1,1,700,2,2022
2,1,600,2,2022
3,2,1000,3,2022
4,2,1100,1,2022
5,3,400,1,2021
6,3,500,1,2021


,salary,month,year,YTD Sum
5,400,1,2021,400
6,500,1,2021,900
0,100,1,2022,100
4,1100,1,2022,1200
1,700,2,2022,700
2,600,2,2022,1300
3,1000,3,2022,1000


In [41]:
import random

def generate_data(num_rows):
    names = ["Alice", "Bob", "Charlie", "David", "Emily", "Frank", "Grace", "Harry", "Isabella", "Jack"]  # Example names
    salaries = range(40000, 120001, 5000)  # Salary range
    data = [(random.choice(names), random.choice(salaries)) for _ in range(num_rows)]
    return data

# Generate 100000 rows of data
list_1 = generate_data(5000000)
list_1[0:2]

[('Grace', 80000), ('Charlie', 65000)]

In [71]:
%%timeit
def find_tuples_above_average(list_1):
   average_salary = sum(i[1] for i in list_1) / len(list_1)
   return [(name, salary) for name, salary in list_1 if salary > average_salary]
high_earners = find_tuples_above_average(list_1)
print(len(high_earners))

2645642
2645642
2645642
2645642
2645642
2645642
2645642
2645642
623 ms ± 91.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [69]:
%%timeit
import numpy as np
# names, salaries = zip(*list_1)
np_names=np.array(names)
np_salaries=np.array(list_1[5][1])
average_salary = np_salaries.mean()
print(len(names[np_salaries>average_salary]))

<magic-timeit>:6: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


5
5
5
5
5
5
5
5
685 ms ± 49.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [61]:
list_1[:][1]

('Charlie', 65000)